In [ ]:
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [ ]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [ ]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [ ]:
vocab = sorted(set(text))
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

def text_to_int(text):
  return np.array([char2idx[c] for c in text])

text_as_int = text_to_int(text)

In [ ]:
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: First Citizen
Encoded: [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:13]))

First Citizen


In [ ]:
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You

OUTPUT
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 


EXAMPLE

INPUT
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 

OUTPUT
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-6.48643635e-03 -6.16371259e-03  9.73703281e-05 ... -4.78683272e-03
    1.51243934e-03 -4.01099632e-03]
  [-6.32165093e-03 -2.50593456e-03 -9.18984879e-05 ... -5.13757532e-03
    5.33785485e-03  1.02609396e-04]
  [-5.84340980e-03  3.38461250e-05 -3.96536605e-04 ... -4.89291362e-03
    8.31080135e-03  3.50308255e-03]
  ...
  [-3.87006905e-04  2.88494723e-03 -8.94172769e-03 ...  3.45854042e-03
    1.16335147e-03 -9.98336123e-04]
  [ 9.28209163e-04 -2.25963304e-03 -1.63321048e-02 ...  6.97530434e-03
    4.09109471e-03 -5.08822501e-04]
  [-5.87529782e-03 -7.55203189e-03 -1.15630748e-02 ...  1.21748331e-03
    6.57962775e-03 -5.05099958e-03]]

 [[ 5.30495681e-03  4.48574370e-04  2.52465252e-03 ... -3.16061708e-03
    5.43312682e-03 -1.14389346e-04]
  [ 4.58460208e-03 -1.06487377e-03 -1.84836809e-03 ...  1.70023972e-03
    2.42939731e-03 -3.22988303e-03]
  [-2.07736017e-03 -4.96585330e-04  2.90052919e-03 ...  5.03088720e-03
   -1.55416608e-03 -3.00702569e-03]
  ...
  [ 5.530

In [ ]:
pred = example_batch_predictions[0]
print(len(pred))
print(pred)

100
tf.Tensor(
[[-6.48643635e-03 -6.16371259e-03  9.73703281e-05 ... -4.78683272e-03
   1.51243934e-03 -4.01099632e-03]
 [-6.32165093e-03 -2.50593456e-03 -9.18984879e-05 ... -5.13757532e-03
   5.33785485e-03  1.02609396e-04]
 [-5.84340980e-03  3.38461250e-05 -3.96536605e-04 ... -4.89291362e-03
   8.31080135e-03  3.50308255e-03]
 ...
 [-3.87006905e-04  2.88494723e-03 -8.94172769e-03 ...  3.45854042e-03
   1.16335147e-03 -9.98336123e-04]
 [ 9.28209163e-04 -2.25963304e-03 -1.63321048e-02 ...  6.97530434e-03
   4.09109471e-03 -5.08822501e-04]
 [-5.87529782e-03 -7.55203189e-03 -1.15630748e-02 ...  1.21748331e-03
   6.57962775e-03 -5.05099958e-03]], shape=(100, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)

65
tf.Tensor(
[-6.4864364e-03 -6.1637126e-03  9.7370328e-05  2.3057193e-03
 -3.9805258e-03 -2.3698667e-05  4.2010853e-03 -3.1025575e-03
 -4.1924557e-03 -2.7411757e-03 -8.4670754e-03 -3.1111056e-03
  1.1848765e-03 -1.8561697e-03  3.0726618e-03  1.7443453e-03
 -2.5102342e-03  4.9893139e-03 -3.3055686e-03 -5.5901352e-03
  5.0351198e-04  1.5547767e-03 -7.2547263e-03 -1.8744769e-03
 -5.1742513e-04  1.1339749e-03  3.6851419e-03  3.1041005e-03
 -5.5878572e-03  1.1287626e-03  5.4616202e-03 -1.1993120e-03
 -2.3480321e-03  3.4788908e-03  1.6353196e-03 -1.4213037e-03
  2.0684318e-03  3.2083679e-03  1.1873455e-03  6.1936360e-03
  4.9810754e-03  5.3288299e-03  1.7957556e-03  2.6574805e-03
 -2.1952745e-03 -2.4874657e-03  9.3412595e-03  1.6059671e-04
  1.0050255e-03 -4.0832686e-04  1.3024595e-03 -1.4772383e-03
 -9.6683030e-04  1.6444469e-03  2.2032941e-03 -1.1217443e-02
 -2.9116124e-03  1.3983968e-03  1.1706385e-03  2.7717315e-03
 -1.5075956e-03  2.2751794e-03 -4.7868327e-03  1.5124393e-03
 -4.010996

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"tpy\nAYIbRruNxSphc:$iZDoNGtKkXP'nLm;luNU-oU$\n-RCtGzybIP:GSeHs-BwcvaDx,PFTd:MNxUE3MgDZkvGko'tffnl\nEL-3"

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 14s 69ms/step - loss: 2.6129
Epoch 2/50
172/172 [==============================] - 13s 70ms/step - loss: 1.9235
Epoch 3/50
172/172 [==============================] - 13s 72ms/step - loss: 1.6673
Epoch 4/50
172/172 [==============================] - 13s 72ms/step - loss: 1.5253
Epoch 5/50
172/172 [==============================] - 13s 71ms/step - loss: 1.4395
Epoch 6/50
172/172 [==============================] - 13s 72ms/step - loss: 1.3790
Epoch 7/50
172/172 [==============================] - 13s 71ms/step - loss: 1.3324
Epoch 8/50
172/172 [==============================] - 13s 71ms/step - loss: 1.2932
Epoch 9/50
172/172 [==============================] - 13s 72ms/step - loss: 1.2568
Epoch 10/50
172/172 [==============================] - 13s 70ms/step - loss: 1.2225
Epoch 11/50
172/172 [==============================] - 13s 71ms/step - loss: 1.1879
Epoch 12/50
172/172 [==============================] - 13s 70ms/step - loss: 1.1530
E

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 800

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: none
noness. Therefore, good friar?

DUKE VINCENTIO:
Go hither to the boornot cry 'D What's that to you?
'Tis not proud one I did but the less,
While I under the thoughts of this dividengears and pretty boyou speak against
Our fire must flee supposes the sin that live?

ROMEO:
Ay, so Gloucester, lady: lest abson your hate,
But but one peor good friar, this is he in his kind
And favour of unruly children's bosoms sillies,
Didouble action, and his son for thee
In the dark backward the Fretzhing nature
I can see her beholding to your royal person, have the grace
As for Bolingbroke to bid his foe.

KING RICHARD II:
Go, Bushy, and Sasting to the crown and orcourteman:
Let them have scope in quiet rest thou off!

First Lord:
O Tullus,--

Secoved CApullet's incruse, and the wood dopose
yet I beherefore yo
